In [1]:
import calendar
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine, text
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.max_row',None)

data_path = os.path.join(os.path.expanduser("~"), "OneDrive","A5","Data")
csv_path = os.path.join(os.path.expanduser("~"), "iCloudDrive")
one_path = os.path.join(os.path.expanduser("~"), "OneDrive","Documents","Data")
osd_path = os.path.join(os.path.expanduser("~"),"OneDrive","Documents","obsidian-git-sync","Data")
dts_path = os.path.join(os.path.expanduser("~"),"Downloads","Datasets")

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2025, 1, 19), datetime.date(2025, 1, 18))

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
#yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2025-01-19
yesterday: 2025-01-17 00:00:00


In [3]:
yesterday = yesterday.date()
a_year_ago = yesterday - timedelta(days=365)
yesterday, a_year_ago

(datetime.date(2025, 1, 17), datetime.date(2024, 1, 18))

### Restart and Run All Cells

### Weekly process or when stataus changes.

In [6]:
pd.read_sql_query('SELECT COUNT(*) AS records FROM sales', conlite)

,records
0,0


In [7]:
sqlDel = text("""DELETE FROM sales""")
rp = conlite.execute(sqlDel)
conlite.commit()
rp.rowcount

0

In [8]:
sql = """
SELECT name
FROM orders 
ORDER BY name
"""
df = pd.read_sql(sql, conlite)

names = df['name'].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'3BBIF', 'AIMIRT', 'AWC', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'PTG', 'RCL', 'SENA', 'SYNEX', 'TFFIF', 'TVO', 'WHAIR', 'WHART'"

In [9]:
len(in_p.split(','))

20

### Get past one year data

In [11]:
sql = """
SELECT name, date, price, qty, maxp, minp 
FROM price 
WHERE date >= '%s' AND name IN (%s) 
ORDER BY name, date"""
sql = sql % (a_year_ago, in_p)
print(sql)


SELECT name, date, price, qty, maxp, minp 
FROM price 
WHERE date >= '2024-01-18' AND name IN ('3BBIF', 'AIMIRT', 'AWC', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'PTG', 'RCL', 'SENA', 'SYNEX', 'TFFIF', 'TVO', 'WHAIR', 'WHART') 
ORDER BY name, date


In [12]:
df = pd.read_sql(sql, const)
df.tail()

,name,date,price,qty,maxp,minp
4810,WHART,2025-01-13,10.1,2927430,10.1,10.0
4811,WHART,2025-01-14,10.1,1222470,10.3,10.0
4812,WHART,2025-01-15,10.1,1387741,10.2,10.0
4813,WHART,2025-01-16,10.1,794700,10.2,10.0
4814,WHART,2025-01-17,10.1,766767,10.2,10.0


In [13]:
file_name = "Yearly-Price-by-Name.csv"
output_file = data_path + '/' + file_name
output_file 

'C:\\Users\\PC1\\OneDrive\\A5\\Data/Yearly-Price-by-Name.csv'

In [14]:
df.set_index("name", inplace=True)
df.to_csv(output_file, header=None)

### Create monitors from orders

In [16]:
sql = """
SELECT name,trade 
FROM orders 
WHERE name IN (%s)
ORDER BY name
"""
sql = sql % in_p
orders = pd.read_sql(sql, conlite)
orders.set_index(['name'],inplace=True)
orders

,trade
name,
3BBIF,B
AIMIRT,B
AWC,S
BCH,S
CPNREIT,B
DIF,B
GVREIT,B
JMART,B
JMT,S


In [17]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name

orders.to_csv(data_file, header=None)
orders.to_csv(output_file)

In [18]:
sql = """
SELECT trade, COUNT(*) AS items 
FROM orders
GROUP BY trade
ORDER BY trade
"""
grp = pd.read_sql(sql, conlite)
grp

,trade,items
0,B,12
1,S,8


### For new orders that never have XXX.html & XXX.CSV

In [20]:
file_name = 'price-uploads.csv'
url = data_path + '\\' + file_name
url

'C:\\Users\\PC1\\OneDrive\\A5\\Data\\price-uploads.csv'

In [21]:
uploads = pd.read_csv(url)
uploads.sort_values(['name'],ascending=[True]).shape

(152, 1)

In [22]:
df_merge = pd.merge(orders, uploads, on='name', how='outer', indicator=True)
df_merge.sort_values(['name'],ascending=[True]).shape

(153, 3)

In [23]:
new_prices = df_merge['_merge'] == 'left_only'
df_merge[new_prices]

,name,trade,_merge
11,AWC,S,left_only


In [24]:
new_prices = df_merge['_merge'] == 'right_only'
df_merge[new_prices].shape

(133, 3)

In [25]:
sql = """
SELECT name, status, market
FROM stocks 
ORDER BY status, name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks.shape

(31, 2)

In [26]:
file_name = "stocks-all.csv"
data_file = data_path + '\\' + file_name
output_file = csv_path + '\\' + file_name
one_file = one_path + '\\' + file_name
data_file

'C:\\Users\\PC1\\OneDrive\\A5\\Data\\stocks-all.csv'

In [27]:
stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(one_file)

In [28]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","O")
ORDER BY name
"""
buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,status
name,
3BBIF,B
AIMIRT,B
CPNREIT,B
DIF,B
GVREIT,B
JMART,B
KCE,B
MCS,B
NER,B


In [29]:
buy_candidates.shape[0]

12

In [30]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("I","S")
ORDER BY name
"""

sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,status
name,
AWC,S
BCH,S
JMT,S
PTG,S
RCL,S
SYNEX,S
TVO,S
WHART,S


In [31]:
sell_candidates.shape[0]

8

In [32]:
conlite.close()
const.close()